In [16]:
import os
import PIL.Image
import PIL.ExifTags
import pandas as pd

In [17]:
path = './test'


In [18]:
for i in os.listdir(path):
    image_path = os.path.join(path, i)
    img = PIL.Image.open(image_path)
    exif_data = img._getexif()
    #print(img.getxmp())
    print(exif_data)

{34853: {0: b'\x02\x03\x00\x00', 1: 'S', 2: (0.0, 27.6583, 0.0), 3: 'W', 4: (80.0, 27.4632, 0.0), 5: b'\x00', 6: 9.7, 7: (17.0, 15.0, 32.0), 8: '11', 16: 'T', 17: 165.53, 18: 'WGS-84\x00  ', 29: '2016:06:01'}, 296: 2, 34665: 19714, 271: 'NIKON', 272: 'COOLPIX AW100\x00 ', 305: 'Microsoft Windows Photo Viewer 6.1.7600.16385', 274: 1, 306: '2016:06:17 14:52:04', 531: 2, 282: 300.0, 283: 300.0, 59932: b'\x1c\xea\x00\x00\x00\x08\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00

In [22]:
df = pd.DataFrame()

for i in os.listdir(path):
    image_path = os.path.join(path, i)
    img = PIL.Image.open(image_path)
    
    #Get EXIF data    
    exif = {
        PIL.ExifTags.TAGS[k]: v
        for k, v in img._getexif().items()
        if k in PIL.ExifTags.TAGS
        }
    print(exif)
    
    date = img._getexif()[36867]
    print(date)
    
    #Get GPS data
    gps={}
    for k, v in exif['GPSInfo'].items():
        geo_tag = PIL.ExifTags.GPSTAGS.get(k)
        gps[geo_tag]=v
 
    #Get Latitude and Longitude
    lat = gps['GPSLatitude']
    long = gps['GPSLongitude']
    
    #Convert to degrees
    lat = float(lat[0]+(lat[1]/60)+(lat[2]/(3600*100)))
    long = float(long[0]+(long[1]/60)+(long[2]/(3600*100)))
    
    #Negative if LatitudeRef:S or LongitudeRef:W
    if gps['GPSLatitudeRef']=='S':
        lat = -lat
    if gps['GPSLongitudeRef']=='W':
        long = -long
        
    #Get DateTime metadata
    
    #print(lat, long)
    coords = {'filename': i, 'latitude': lat, 'longitude': long, 'date': date}
    
    #print(img.getxmp().items())
    
    df_loc = pd.DataFrame(coords, index=[0])
    df = pd.concat([df, df_loc], ignore_index=True)
    

{'GPSInfo': {0: b'\x02\x03\x00\x00', 1: 'S', 2: (0.0, 27.6583, 0.0), 3: 'W', 4: (80.0, 27.4632, 0.0), 5: b'\x00', 6: 9.7, 7: (17.0, 15.0, 32.0), 8: '11', 16: 'T', 17: 165.53, 18: 'WGS-84\x00  ', 29: '2016:06:01'}, 'ResolutionUnit': 2, 'ExifOffset': 19714, 'Make': 'NIKON', 'Model': 'COOLPIX AW100\x00 ', 'Software': 'Microsoft Windows Photo Viewer 6.1.7600.16385', 'Orientation': 1, 'DateTime': '2016:06:17 14:52:04', 'YCbCrPositioning': 2, 'XResolution': 300.0, 'YResolution': 300.0, 'ExifVersion': b'0230', 'ComponentsConfiguration': b'\x01\x02\x03\x00', 'CompressedBitsPerPixel': 4.0, 'DateTimeOriginal': '2016:06:01 12:14:33', 'DateTimeDigitized': '2016:06:01 12:14:33', 'ShutterSpeedValue': 8.965784, 'ApertureValue': 5.926948, 'ExposureBiasValue': 0.0, 'MaxApertureValue': 3.9, 'MeteringMode': 5, 'LightSource': 0, 'Flash': 16, 'FocalLength': 5.0, 'ColorSpace': 1, 'ExifImageWidth': 3456, 'SceneCaptureType': 1, 'ExifImageHeight': 4608, 'Contrast': 0, 'Saturation': 0, 'Sharpness': 0, 'SubjectD

KeyError: 270

In [21]:
df.head()

,filename,latitude,longitude,date
0,DSCN3211.JPG,-0.460972,-80.457720,2016:06:01 12:14:33
1,DSCN3212.JPG,-0.460967,-80.457733,2016:06:01 12:14:41
2,DSCN3213.JPG,-0.460950,-80.457728,2016:06:01 12:14:59
3,DSCN3214.JPG,-0.461338,-80.457572,2016:06:01 12:17:04
4,DSCN3536.JPG,0.069645,-80.055800,2016:06:02 15:31:41


In [15]:
df.to_csv('exif_data.csv', index=False)